# Web Crawling Project 2
## '뉴스 기사 크롤링하기'

In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# naver 뉴스

In [250]:
driver = webdriver.Chrome('chromedriver.exe')

driver.get('http://naver.com')

driver.find_element_by_id('query').send_keys('동성제약')
driver.find_element_by_id('search_btn').click()

driver.find_element_by_css_selector('div.lnb_menu').find_element_by_link_text(
    '뉴스').click()

driver.find_element_by_id('_search_option_btn').click()
menu = driver.find_elements_by_css_selector('ul.option_menu > li.menu')
menu[2].click()
menu[2].find_element_by_link_text('제목').click()

current_window = driver.current_window_handle

data = {'title': [], 'date': [], 'company': [], 'context': [], 'reporter': []}
for i in range(50):
    for naver_news in driver.find_elements_by_link_text('네이버뉴스'):
        naver_news.click()
        driver.switch_to.window(driver.window_handles[1])
        html = driver.page_source
        soup = bs(html, 'html.parser')
        data['title'].append(soup.select('h3.tts_head')[0].text.strip())
        data['date'].append(soup.select('span.t11')[0].text.strip())
        soup.find('div', id='articleBodyContents').script.extract()
        data['context'].append(
            soup.find('div', id='articleBodyContents').text.strip())
        driver.close()
        driver.switch_to.window(current_window)
    driver.find_element_by_link_text('다음페이지').click()

In [251]:
for i in range(50):
    for naver_news in driver.find_elements_by_link_text('네이버뉴스'):
        naver_news.click()
        driver.switch_to.window(driver.window_handles[1])
        html = driver.page_source
        soup = bs(html, 'html.parser')
        data['title'].append(soup.select('h3.tts_head')[0].text.strip())
        data['date'].append(soup.select('span.t11')[0].text.strip())
        soup.find('div', id='articleBodyContents').script.extract()
        data['context'].append(
            soup.find('div', id='articleBodyContents').text.strip())
        driver.close()
        driver.switch_to.window(current_window)
    driver.find_element_by_link_text('다음페이지').click()

In [348]:
driver.switch_to.window(current_window)

In [349]:
%%time

for i in range(120):    
    for naver_news in driver.find_elements_by_link_text('네이버뉴스'):
        naver_news.click()
        driver.switch_to.window(driver.window_handles[1])
        html = driver.page_source
        soup = bs(html, 'html.parser')
        if len(soup.select('h3.tts_head')) == 0:
            driver.close()
            driver.switch_to.window(current_window)
            break
        data['title'].append(soup.select('h3.tts_head')[0].text.strip())
        data['date'].append(soup.select('span.t11')[0].text.strip())
        soup.find('div', id = 'articleBodyContents').script.extract()
        data['context'].append(soup.find('div', id = 'articleBodyContents').text.strip())
        driver.close()
        driver.switch_to.window(current_window)
    driver.find_element_by_link_text('다음페이지').click()

Wall time: 36min 53s


https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=022&aid=0002671699

이 놈에서 자꾸 오류가 발생하는데.... 뭔지를 모르겟다

# 발생한 오류
### 1. 인터넷 뉴스 회사마다 html형식이 달라 크롤링에 제약이 생겼다.
 - 해결 못 함

### 2. pd.read_csv() 과정에서 'Initializing from file failed' 오류가 발생
 - pandas의 자체 버그이며 해결방법으로는 engine = 'python' 옵션을 추가해주면 된다.
 
### 3. for구문 중간에 자꾸 list Index out of range 오류가 발생
 - 이거는 단지 select된 element내에 아무것도 없었기 때문에 [0]에서 오류가 발생
 - if구문을 통해 조건을 걸어 무시할 수 있도록 조치함

# 새로 배운 것
### 1. Multiple class인 element 찾기
 - css_selector(.class.class)

### 2. BeautifulSoup를 통해 parsing된 문서에서 id로 찾기(select 말고)
 - soup.find('tag', id = '')
 - 참고https://stackoverflow.com/questions/25614702/get-contents-of-div-by-id-with-beautifulsoup
 
### 3. 글 중간에 있는 특정 tag를 제외한 상태로 긁어오기
 - soup.find().tag.extract() : tag를 extract()한 상태가 된다.(soup 안에서 해당 tag가 삭제됨)
 - 참고https://stackoverflow.com/questions/48877343/exclude-tags-with-beautifulsoup
 - 제외한다는 말을 'exclude'라고 표현함
 
### 4. DataFrame 내에 중복되는 row 삭제하기
 - pd.drop_duplicates(column_label, keep = 'first')
 - https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop_duplicates.html